Veri işleme ve analizi için pandas kütüphanesi kullanılacaktır. Pandas kütüphanesi pd rumuzu ile kısaltılmıştır.

In [1]:
import pandas as pd

Verilerimiz Google Drive (bulut) üzerinde olduğu için Google Colab ile Google Drive arasında bağlantı kurulacaktır.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Google Drive içinde Colab Notebooks klasörünün içinde TDDI2022 isimli klasörde verilerimizi sakladığımız için bu klasöre erişilecektir.

***NOT:*** Bu klasörü daha önce oluşturmadıysanız %md TDDI2022 komutu veya Google Drive Ara Yüzü ile klasörü oluşturunuz!

In [ ]:
%cd drive/
%cd MyDrive/
%cd Colab Notebooks/
# Google Drive içinde Colab Notebooks klasörünün içinde TDDI2022 diye bir klasör oluşturduktan sonra o klasöre erişiyoruz.
# Bu klasörü daha önce oluşturmadıysanız %md TDDI2022 komutu veya Google Drive Ara Yüzü ile klasörü oluşturunuz!
%cd TDDI2022/

# Klasörün içeriğinin görülmesi için
%ls

Veri İskeleti TDDI2022 klasöründe bulunan yarışma kapsamında yarışmacılara sağlanan **kanunum-nlp-doc-analysis-dataset.csv** dosyasının içindeki verileri pandas ile okuyarak oluşturulacaktır.

Veri iskeleti oluşturulduktan sonra veri iskeleti ile ilgili bilgi ekrana yazdırılacaktır. Tam özetin yazdırılıp yazdırılmayacağı durumunu *verbose=True* (yazdırılsın) parametresi ile kontrol edilecektir. 

In [ ]:
df = pd.read_csv('kanunum-nlp-doc-analysis-dataset.csv')
df.info(verbose=True)

Veri İskeletinde **kategori** sütununun / alanının kaç gruptan ve bu grupların kaç satırdan / kayıttan oluştuğu ekrana yazdırılacaktır.

In [ ]:
print(df.groupby('kategori').size())

Veriler üzerinde yaptığımız ön çalışmada (ekrana data_text sütunundan / alanından bir kaç satır yazdırılmış ve içerikleri incelenmiştir) veri kümesinin data_text sütunundaki / alanındaki içerik iki parçaya ayrılabileceği görülmüştür. Böylelikle data_text sütunu / alanı işlenirken belgeye ait bilgiler birinci parçada belgenin içeriği ikinci parçada görülebilir. Tüm data_text kayıtlarında yer alan ortak bir noktanın varlığını kontrol etmek için aşağıdaki kod yardımı ile resmi belgelerin hepsinde 'Madde 1 -' verisi var mı?

In [ ]:
kayitSayisi, alanSayisi = df.shape
say = 0
satirToplami, sutunToplami = df.shape
for satir in range(satirToplami):
  if df['data_text'][satir].find('Madde 1 -'):
    say+=1
if say==kayitSayisi:
  print(say,'=',kayitSayisi,'yani data_text alanındaki toplam kayıt sayısı ve her bir kayıtta "Madde 1 -" verisini içeren kayıt sayısı birbirine eşittir.')
else:
  print('Tüm kayıtlarda aranan veri bulunamamıştır.')

Yukarıdaki bulgular doğrultusunda örnek bir kaydı ikiye bölerek **belge ile ilgili verileri toplayabileceğimiz içerik kısmı** görelim. Yine bu içerik kısmında kaç adet madde olduğunu görmek için madde_sayisi alanındaki veriyi de ekrana yazdıralım.

In [ ]:
satir = 1881
print('Dokümandaki Madde Sayısı:',df['madde_sayisi'][satir],'\n',50*'-','\nİçerik kısmı:',df['data_text'][satir][df['data_text'][satir].find('Madde 1'):],sep='')

1881 indis numaralı kayıtta 58 adet madde olduğunu gördük. Şimdi kural bazlı olarak bu kayıt üzerinde madde saydıracağız.

Kuralımız: - Maddeyi bul madde bir önceki maddenin ardışık olarak takip ettiği madde ise örneğin bulduğumuz madde 5. madde olsun ve şimdiye kadar 4 madde saymış olalım 5. maddeyi madde olarak sayacak ve bu kayıtla blirlikte 5 madde sayılmış olacaktır. Aynı durum ek madde ve geçici maddeler için de uygulanabilirdir.

Şimdi kuralımız gerçekten anlatığımız gibi çalışıp 1881 indisli kayıtta 58 madde bulabilecek mi inceleyelim.

In [ ]:
# data_text alanının 1881 indisli satırında madde sayamaya çalışalım!
satir = 1881

ilkMaddeninBasladigiYer = df['data_text'][satir].find('Madde 1')
verininBuyuklugu = len(df['data_text'][satir])
# İlk maddenin başladığı yerden itibaren belirli aralıklarla MADDE SAYMAYA başlıyoruz!
# İlk değer olarak maddeSayisi değişkenimize madde yoktur manasında 0 'SIFIR' veriyoruz.
maddeSayisi = 0

# aramaya baslayacağımız yeri belirliyoruz.,
aramaYapilacakYerinBasi = ilkMaddeninBasladigiYer

while (aramaYapilacakYerinBasi<verininBuyuklugu):
  print('maddeSayisi:',maddeSayisi)
  bulunanSayi = ''
  alinanKesit = df['data_text'][satir][aramaYapilacakYerinBasi:]
  maddeninBaslayabilecegiYer = alinanKesit.find('Madde')
  print('maddeninBaslayabilecegiYer:',maddeninBaslayabilecegiYer)
  tireninBaslayabilecegiYer = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('–')
  print('tireninBaslayabilecegiYer:',tireninBaslayabilecegiYer)
  if (tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=10:
    maddeninBaslayabilecegiYer+=5
    while maddeninBaslayabilecegiYer<tireninBaslayabilecegiYer:
      maddeninBaslayabilecegiYer+=1
      bulunanSayiyiOlusturacakParca = alinanKesit[maddeninBaslayabilecegiYer]
      print(bulunanSayiyiOlusturacakParca)
      if bulunanSayiyiOlusturacakParca in '0123456789':
        bulunanSayi+=bulunanSayiyiOlusturacakParca
      else:
            break
    if bulunanSayi!='':
      if (int(bulunanSayi)<=(maddeSayisi+2)) and (int(bulunanSayi)>=(maddeSayisi-2)):
        maddeSayisi += int(bulunanSayi)-maddeSayisi
  aramaYapilacakYerinBasi+=maddeninBaslayabilecegiYer+5
print(satir,'indis numarasına sahip kayıtta olması gereken madde sayısı:',df['madde_sayisi'][satir])
print(satir,'indis numarasına sahip kayıtta kural bazlı yöntemimizin bulduğu madde sayısı:',maddeSayisi)

Kuralımızın çalıştığını 1881 indis sayılı belge için görmüş olduk.

# **ÇIKARIM:**
- Yarışma kapsamında sağlanan veri kümesinin data_text alanında "Madde 1 -" verisine sahip olmayan kayıt/satır yoktur.
- 'Madde' kelimesi ile '-' karakteri arasında 3 basamaklı olacak şekilde sayı varsa bunu mevcut fonksiyonumuz ile tespit edebiliyoruz.
- data_text alanında farklı kayıtlara bakıldığında ve yarışma kapsamında yarışmacılara iletilen README.md dosyası incelendiğinde **geçici maddeler**, **ek maddeler** ve **işlenmeyen hükümlerin maddeleri** gibi maddelerin de olduğu ve madde sayısına etki ettikleri görülmüştür.

In [ ]:
# data_text alanında mevcut yöntemimizle madde sayılarını kontrol edelim.
# Olması gereken ile tahmin ettiğimiz değerlerin birbirlerine eşit olmadığı belge sayısını ve türünü ortaya çıkartalım.

satirToplami, sutunToplami = df.shape

kategorilerimiz = df["kategori"].value_counts().index.values.tolist()
kategoridekiToplamBelgeSayisi = df["kategori"].value_counts().tolist()
kategoridekiHataliBelgeSayisi = [0] * len(kategorilerimiz)

for ilgiliKayit in range(satirToplami):
  # İlk değer olarak maddeSayisi değişkenimize madde yoktur manasında 0 'SIFIR' veriyoruz.
  maddeSayisi = 0
  # İlk maddenin başladığı yerden itibaren belirli aralıklarla MADDE SAYMAYA başlıyoruz!
  ilkMaddeninBasladigiYer = df['data_text'][ilgiliKayit].find('Madde 1')
  if ilkMaddeninBasladigiYer==-1:
    continue
  verininBuyuklugu = len(df['data_text'][ilgiliKayit])

  # aramaya baslayacağımız yeri belirliyoruz.,
  aramaYapilacakYerinBasi = ilkMaddeninBasladigiYer

  while (aramaYapilacakYerinBasi<verininBuyuklugu):
    bulunanSayi = ''
    alinanKesit = df['data_text'][ilgiliKayit][aramaYapilacakYerinBasi:]
    maddeninBaslayabilecegiYer = alinanKesit.find('Madde')
    tireninBaslayabilecegiYer = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('–')
    if (tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=10:
      maddeninBaslayabilecegiYer+=5
      while maddeninBaslayabilecegiYer<tireninBaslayabilecegiYer:
        maddeninBaslayabilecegiYer+=1
        bulunanSayiyiOlusturacakParca = alinanKesit[maddeninBaslayabilecegiYer]
        if bulunanSayiyiOlusturacakParca in '0123456789':
          bulunanSayi+=bulunanSayiyiOlusturacakParca
        else:
              break
      if bulunanSayi!='':
        if (int(bulunanSayi)<=(maddeSayisi+2)) and (int(bulunanSayi)>=(maddeSayisi-2)):
          maddeSayisi += int(bulunanSayi)-maddeSayisi
    aramaYapilacakYerinBasi+=maddeninBaslayabilecegiYer+5
  if maddeSayisi != df['madde_sayisi'][ilgiliKayit]:
    kategoridekiHataliBelgeSayisi[kategorilerimiz.index(df["kategori"][ilgiliKayit])] += 1

for i in range(len(kategorilerimiz)):
  print(kategorilerimiz[i],':',kategoridekiHataliBelgeSayisi[i],'/',kategoridekiToplamBelgeSayisi[i],'=',round(kategoridekiHataliBelgeSayisi[i]/kategoridekiToplamBelgeSayisi[i],3))
print('Toplam Hata:',sum(kategoridekiHataliBelgeSayisi))

# **ÇIKARIM:**

* Kural bazlı madde sayısı bulma yöntemimiz belgelerdeki madde sayısını **0.76** başarı oranında doğru tespit etmektedir. **3159** belgenin madde sayısını doğru olarak bulmakla birlikte **983** belgenin madde sayısını da olması gereken madde sayısından farklı bir sayıda bulmuştur.
* Kural bazlı madde sayısı bulma yöntemimiz **Özelge** kategorisinde **hatasız** çalışmaktadır.
* Kural bazlı madde sayısı bulma yöntemimiz **Komisyon Raporu** (12/500) ve **Resmi Gazete** (2/546) kategorilerinde **istisnai hata** ile çalışmaktadır.
* Kural bazlı madde sayısı bulma yöntemimiz **Kanun Hükmünde Kararname** (24/91) ve **Cumhurbaşkanlığı Kararnamesi** (21/54) kategorilerinde **hatalı** çalıştığı tespit edilmiştir.
* Mevcut yöntemimizin Tebliğ, Yönetmelik, Kanun ve Tüzük kategorilerinde 0.46 - 0.57 arası hata oranında başka bir değişle kabul edilemez büyüklükte bir hata oranında çalıştığı tespit edilmiştir. Bu dört kategorinin içindeki verileri inceleyip yöntemimizi güncelleyeceğiz.

In [ ]:
# data_text alanında mevcut yöntemimizi GEÇİCİ MADDELERİ ve EK MADDELERİ göz önüne alarak madde sayısı hangi doğrulukta bulduğunu kontrol edelim.
# Olması gereken ile tahmin ettiğimiz değerlerin birbirlerine eşit olmadığı belge sayısını ve türünü yeniden hesaplayalım.

satirToplami, sutunToplami = df.shape

kategorilerimiz = df["kategori"].value_counts().index.values.tolist()
kategoridekiToplamBelgeSayisi = df["kategori"].value_counts().tolist()
kategoridekiHataliBelgeSayisi = [0] * len(kategorilerimiz)

for ilgiliKayit in range(satirToplami):

  # İlk değer olarak maddeSayisi değişkenimize madde yoktur manasında 0 'SIFIR' veriyoruz.
  maddeSayisi = 0
  # YÖNTEMİMİZİ GELİŞTİRİYORUZ
  geciciMaddeSayisi = 0 # Geçici Madde 1 gibi verilen ek madde
  ekMaddeSayisi = 0 # Madde içinde 10/a gibi verilen ek madde
  ekMaddeSayisi2 = 0 # Ek Madde 1 gibi verilen ek madde
  # İlk maddenin başladığı yerden itibaren belirli aralıklarla MADDE SAYMAYA başlıyoruz!
  ilkMaddeninBasladigiYer = df['data_text'][ilgiliKayit].find('Madde 1')
  if ilkMaddeninBasladigiYer==-1:
    continue
  verininBuyuklugu = len(df['data_text'][ilgiliKayit])

  # Aramaya baslayacağımız yeri belirliyoruz.
  aramaYapilacakYerinBasi = ilkMaddeninBasladigiYer

  while (aramaYapilacakYerinBasi<verininBuyuklugu):
    bulunanSayi = ''
    alinanKesit = df['data_text'][ilgiliKayit][aramaYapilacakYerinBasi:]
    maddeninBaslayabilecegiYer = alinanKesit.find('Madde')
    tireninBaslayabilecegiYer = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('–')
    if (tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=10:
      maddeninBaslayabilecegiYer+=5
      while maddeninBaslayabilecegiYer<tireninBaslayabilecegiYer:
        maddeninBaslayabilecegiYer+=1
        bulunanSayiyiOlusturacakParca = alinanKesit[maddeninBaslayabilecegiYer]
        if bulunanSayiyiOlusturacakParca in '0123456789':
          bulunanSayi+=bulunanSayiyiOlusturacakParca
        else:
              break
      if bulunanSayi!='':
        if (int(bulunanSayi)<=(maddeSayisi+2)) and (int(bulunanSayi)>=(maddeSayisi-2)):
          maddeSayisi += int(bulunanSayi)-maddeSayisi
        elif int(bulunanSayi)<geciciMaddeSayisi+2:
          geciciMaddeSayisi += int(bulunanSayi)-geciciMaddeSayisi
        elif int(bulunanSayi)<ekMaddeSayisi2+2:
          ekMaddeSayisi2 += int(bulunanSayi)-ekMaddeSayisi2
    else:
      ekMaddeninYeri = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('/')
      if (ekMaddeninYeri<=(tireninBaslayabilecegiYer-2)) and ((tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=12):
        ekMaddeSayisi += 1
    aramaYapilacakYerinBasi+=maddeninBaslayabilecegiYer+5
  maddeSayisi+=(geciciMaddeSayisi+ekMaddeSayisi+ekMaddeSayisi2)
  if maddeSayisi != df['madde_sayisi'][ilgiliKayit]:
    kategoridekiHataliBelgeSayisi[kategorilerimiz.index(df["kategori"][ilgiliKayit])] += 1

for i in range(len(kategorilerimiz)):
  print(kategorilerimiz[i],':',kategoridekiHataliBelgeSayisi[i],'/',kategoridekiToplamBelgeSayisi[i],'=',round(kategoridekiHataliBelgeSayisi[i]/kategoridekiToplamBelgeSayisi[i],3))
print('Toplam Hata:',sum(kategoridekiHataliBelgeSayisi))

# **ÇIKARIM:**

* GEÇİCİ MADDELERİ ve EK MADDELERİ göz önüne alarak güncellediğimiz kural bazlı madde sayısı bulma yöntemimiz belgelerdeki madde sayısını **0.86** başarı oranında doğru tespit etmektedir. **3550** belgenin madde sayısını doğru olarak bulmakla birlikte **592** belgenin madde sayısını da olması gereken madde sayısından farklı bir sayıda bulmuştur.
* Güncellenmiş kural bazlı madde sayısı bulma yöntemimiz yine **Özelge** kategorisinde **hatasız** çalışmaktadır.
* Güncellenmiş kural bazlı madde sayısı bulma yöntemimiz **Komisyon Raporu** kategorisinde hata oranını 12 hatadan 14 hataya çıkartarak (14/500) ve **Resmi Gazete** kategorisinde bir önceki yöntem bulgularını değiştirmeden (2/546) **istisnai hata** ile çalışmaktadır.
* Güncellenmiş kural bazlı madde sayısı bulma yöntemimiz **Kanun Hükmünde Kararname** kategorisinde hata oranını 24 hatadan 12 hataya düşürerek (12/91) ve **Cumhurbaşkanlığı Kararnamesi** kategorisinde hata oranını 21 hatadan 6 hataya düşürerek (6/54) kategorilerinde **hatalı** çalıştığı tespit edilmiştir.
* Güncellenmiş kural bazlı madde sayısı bulma yöntemimiz **Yönetmelik** kategorisinde önemli bir düşüş sağlamıştır. Hata oranını 274 hatadan 48 hataya düşürerek (48/597) **istisnai hata** ile çalışmaktadır.
* Güncellenmiş kural bazlı madde sayısı bulma yöntemimiz **Kanun** kategorisinde de düşüş sağlamıştır. Hata oranını 250 hatadan 132 hataya düşürerek (132/514) **hatalı** çalışmaya devam etmektedir.
* Güncellenmiş kural bazlı madde sayısı bulma yöntemimiz **Tüzük** kategorisinde de düşüş sağlamıştır. Hata oranını 45 hatadan 23 hataya düşürerek (23/82) **hatalı** çalışmaya devam etmektedir.
* Mevcut yöntemimizin **Tebliğ** kategorisindeki 0.57 hata oranında başka bir değişle kabul edilemez büyüklükte bir hata oranında çalışmaya devam etmektedir Tebliğ kategorisine odaklanarak verileri inceleyip yöntemimizi güncelleyeceğiz.

In [ ]:
# Tebliğ Kategorisi odaklı incelememiz ve kural bazlı yöntemimizdeki iyileştirmelerimize başlıyoruz
satir = 3000
print('Doküman Türü:',df['kategori'][satir],'\n','Dokümandaki Madde Sayısı:',df['madde_sayisi'][satir],'\n',50*'-','\nİçerik kısmı:',df['data_text'][satir][df['data_text'][satir].find('Madde 1'):],sep='')

In [ ]:
df["madde_sayisi"].value_counts()

# **ÖNEMLİ BULGU**
* Yarışma kapsamında kullandığımız veri kümesinde madde_sayisi alanında 4142 kayıt dolu olarak görülmektedir.
* Neden tebliğ kategorisinde büyük oranda hata yaptığımızı araştırmak için herhangi bir kayıt olan 3000 indis numaralı kayıda baktığımızda madde sayısının 7 olması gerekirken 0 olarak doldurulduğunu fark ettik.
* 4142 kayıt arasında kaç tanesinin madde sayisi 0 olarak girildiğine baktığımızda 2804 kayıdın madde sayısının 0 olarak girildiğini gördük.

# Madde Sayısı 0 olan kayıtların güncellenmiş kural bazlı yöntemimizle tespit ettiğimiz madde sayısı ile doldurulmasına karar verdik!

In [ ]:
# Güncel kural bazlı yöntemimiz ile hem madde sayısını bulacağız hem de eğer madde sayısı veri kümesinde 0 olarak girilmişse o veriyi bizim kural bazlı yöntemimiz ile bulduğumuz sayı ile dolduralım. 

satirToplami, sutunToplami = df.shape

kategorilerimiz = df["kategori"].value_counts().index.values.tolist()
kategoridekiToplamBelgeSayisi = df["kategori"].value_counts().tolist()
kategoridekiHataliBelgeSayisi = [0] * len(kategorilerimiz)

for ilgiliKayit in range(satirToplami):

  # İlk değer olarak maddeSayisi değişkenimize madde yoktur manasında 0 'SIFIR' veriyoruz.
  maddeSayisi = 0
  # YÖNTEMİMİZİ GELİŞTİRİYORUZ
  geciciMaddeSayisi = 0 # Geçici Madde 1 gibi verilen ek madde
  ekMaddeSayisi = 0 # Madde içinde 10/a gibi verilen ek madde
  ekMaddeSayisi2 = 0 # Ek Madde 1 gibi verilen ek madde
  # İlk maddenin başladığı yerden itibaren belirli aralıklarla MADDE SAYMAYA başlıyoruz!
  ilkMaddeninBasladigiYer = df['data_text'][ilgiliKayit].find('Madde 1')
  if ilkMaddeninBasladigiYer==-1:
    continue
  verininBuyuklugu = len(df['data_text'][ilgiliKayit])

  # Aramaya baslayacağımız yeri belirliyoruz.
  aramaYapilacakYerinBasi = ilkMaddeninBasladigiYer

  while (aramaYapilacakYerinBasi<verininBuyuklugu):
    bulunanSayi = ''
    alinanKesit = df['data_text'][ilgiliKayit][aramaYapilacakYerinBasi:]
    maddeninBaslayabilecegiYer = alinanKesit.find('Madde')
    tireninBaslayabilecegiYer = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('–')
    if (tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=10:
      maddeninBaslayabilecegiYer+=5
      while maddeninBaslayabilecegiYer<tireninBaslayabilecegiYer:
        maddeninBaslayabilecegiYer+=1
        bulunanSayiyiOlusturacakParca = alinanKesit[maddeninBaslayabilecegiYer]
        if bulunanSayiyiOlusturacakParca in '0123456789':
          bulunanSayi+=bulunanSayiyiOlusturacakParca
        else:
              break
      if bulunanSayi!='':
        if (int(bulunanSayi)<=(maddeSayisi+2)) and (int(bulunanSayi)>=(maddeSayisi-2)):
          maddeSayisi += int(bulunanSayi)-maddeSayisi
        elif int(bulunanSayi)<geciciMaddeSayisi+2:
          geciciMaddeSayisi += int(bulunanSayi)-geciciMaddeSayisi
        elif int(bulunanSayi)<ekMaddeSayisi2+2:
          ekMaddeSayisi2 += int(bulunanSayi)-ekMaddeSayisi2
    else:
      ekMaddeninYeri = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('/')
      if (ekMaddeninYeri<=(tireninBaslayabilecegiYer-2)) and ((tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=12):
        ekMaddeSayisi += 1
    aramaYapilacakYerinBasi+=maddeninBaslayabilecegiYer+5
  maddeSayisi+=(geciciMaddeSayisi+ekMaddeSayisi+ekMaddeSayisi2)
  if df['madde_sayisi'][ilgiliKayit]==0:
    df.loc[ilgiliKayit,'madde_sayisi'] = maddeSayisi
  else:
    if maddeSayisi != df['madde_sayisi'][ilgiliKayit]:
      kategoridekiHataliBelgeSayisi[kategorilerimiz.index(df["kategori"][ilgiliKayit])] += 1

for i in range(len(kategorilerimiz)):
  print(kategorilerimiz[i],':',kategoridekiHataliBelgeSayisi[i],'/',kategoridekiToplamBelgeSayisi[i],'=',round(kategoridekiHataliBelgeSayisi[i]/kategoridekiToplamBelgeSayisi[i],3))
print('Toplam Hata:',sum(kategoridekiHataliBelgeSayisi))

# **Çıkarım:**
* Madde Sayısı 0 olarak doldurulmuş verileri düzelttiğimizde hata sayımız 221 hataya düşmüştür. Başka bir değişle güncel kural bazlı sistemimiz **0.95** başarı oranı ile belgedeki madde sayısını doğru bir şekilde tespit etmektedir.

* Güncellenmiş kural bazlı madde sayısı bulma yöntemimiz madde sayısı 0 girilen kayıtları bizim bulduğumuz sayı ile güncelledikten sonra yine **Özelge** kategorisinde **hatasız** çalışmaktadır.
* (ÖNEMLİ BULGU) Güncellenmiş kural bazlı madde sayısı bulma yöntemimiz madde sayısı 0 girilen kayıtları bizim bulduğumuz sayı ile güncelledikten sonra artık **Tebliğ**, **Resmi Gazete**, **Komisyon Raporu** ve **Genelge** kategorilerinde de **hatasız** çalışmaktadır.
* Güncellenmiş kural bazlı madde sayısı bulma yöntemimiz madde sayısı 0 girilen kayıtları bizim bulduğumuz sayı ile güncelledikten sonra **Yönetmelik** kategorisinde hata oranında **DEĞİŞİKLİK OLMADAN** (48/597) **hatalı / istisnai hata** ile çalışmaktadır.
* Güncellenmiş kural bazlı madde sayısı bulma yöntemimiz madde sayısı 0 girilen kayıtları bizim bulduğumuz sayı ile güncelledikten sonra **Kanun** kategorisinde hata oranında **DEĞİŞİKLİK OLMADAN** (132/514) **hatalı** çalışmaya devam etmektedir.
* Güncellenmiş kural bazlı madde sayısı bulma yöntemimiz madde sayısı 0 girilen kayıtları bizim bulduğumuz sayı ile güncelledikten sonra **Kanun Hükmünde Kararname** kategorisinde hata oranında **DEĞİŞİKLİK OLMADAN** (12/91) **hatalı** çalışmaya devam etmektedir.
* Güncellenmiş kural bazlı madde sayısı bulma yöntemimiz madde sayısı 0 girilen kayıtları bizim bulduğumuz sayı ile güncelledikten sonra **Tüzük** kategorisinde hata oranında **DEĞİŞİKLİK OLMADAN** (23/82) **hatalı** çalışmaya devam etmektedir.
* Güncellenmiş kural bazlı madde sayısı bulma yöntemimiz madde sayısı 0 girilen kayıtları bizim bulduğumuz sayı ile güncelledikten sonra **Cumhurbaşkanlığı Kararnamesi** kategorisinde hata oranında **DEĞİŞİKLİK OLMADAN** (6/54) **hatalı** çalışmaya devam etmektedir.

* Bu 221 kayıda bakarak neyi madde olarak saymadığımızı kontrol edeceğiz. Bu kontrolü de önce maddeleri eksik mi sayıyoruz yoksa fazla mı sayıyoruz diye kontrol edeceğiz.

In [ ]:
# 221 Kayıtta EKSİK saydığımız madde mi var yoksa fazla saydığımız madde mi var?
# Güncel kural bazlı yöntemimiz ile hem madde sayısını bulacağız hem de eğer madde sayısı veri kümesinde 0 olarak girilmişse o veriyi bizim kural bazlı yöntemimiz ile bulduğumuz sayı ile dolduralım. 

maddeSayisiEksikSayilanBelgeSayisi = 0
birFarkOlanBelgeSayisi = 0
maddeSayisiFazlaSayilanBelgeSayisi = 0

satirToplami, sutunToplami = df.shape

kategorilerimiz = df["kategori"].value_counts().index.values.tolist()
kategoridekiToplamBelgeSayisi = df["kategori"].value_counts().tolist()
kategoridekiHataliBelgeSayisi = [0] * len(kategorilerimiz)

for ilgiliKayit in range(satirToplami):
  if df["kategori"][ilgiliKayit] in ["Özelge","Tebliğ","Resmi Gazete","Komisyon Raporu","Genelge"]:
    pass
  else:
    # İlk değer olarak maddeSayisi değişkenimize madde yoktur manasında 0 'SIFIR' veriyoruz.
    maddeSayisi = 0
    # YÖNTEMİMİZİ GELİŞTİRİYORUZ
    geciciMaddeSayisi = 0 # Geçici Madde 1 gibi verilen ek madde
    ekMaddeSayisi = 0 # Madde içinde 10/a gibi verilen ek madde
    ekMaddeSayisi2 = 0 # Ek Madde 1 gibi verilen ek madde
    hataliMaddeSayisi = 0
    # İlk maddenin başladığı yerden itibaren belirli aralıklarla MADDE SAYMAYA başlıyoruz!
    ilkMaddeninBasladigiYer = df['data_text'][ilgiliKayit].find('Madde 1')
    if ilkMaddeninBasladigiYer==-1:
      continue
    verininBuyuklugu = len(df['data_text'][ilgiliKayit])

    # Aramaya baslayacağımız yeri belirliyoruz.
    aramaYapilacakYerinBasi = ilkMaddeninBasladigiYer

    while (aramaYapilacakYerinBasi<verininBuyuklugu):
      bulunanSayi = ''
      alinanKesit = df['data_text'][ilgiliKayit][aramaYapilacakYerinBasi:]
      maddeninBaslayabilecegiYer = alinanKesit.find('Madde')
      tireninBaslayabilecegiYer = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('–')
      if (tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=10:
        eskiMaddeIndex = maddeninBaslayabilecegiYer
        maddeninBaslayabilecegiYer+=5
        while maddeninBaslayabilecegiYer<tireninBaslayabilecegiYer:
          maddeninBaslayabilecegiYer+=1
          bulunanSayiyiOlusturacakParca = alinanKesit[maddeninBaslayabilecegiYer]
          if bulunanSayiyiOlusturacakParca in '0123456789ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ':
            bulunanSayi+=bulunanSayiyiOlusturacakParca
          else:
                break
        if bulunanSayi!='':
           try:
            if (int(bulunanSayi)<=(maddeSayisi+2)) and (int(bulunanSayi)>=(maddeSayisi-2)):
              maddeSayisi += 1
            elif int(bulunanSayi)<geciciMaddeSayisi+2:
              geciciMaddeSayisi += 1
            elif int(bulunanSayi)<ekMaddeSayisi2+2:
              ekMaddeSayisi2 += 1
           except:
             hataliMaddeSayisi += 1
        elif alinanKesit[eskiMaddeIndex-7:eskiMaddeIndex-1] == "Geçici":
          geciciMaddeSayisi += 1
        
      else:
        ekMaddeninYeri = maddeninBaslayabilecegiYer+alinanKesit[maddeninBaslayabilecegiYer:].find('/')
        if (ekMaddeninYeri<=(tireninBaslayabilecegiYer-2)) and ((tireninBaslayabilecegiYer-maddeninBaslayabilecegiYer)<=12):
          ekMaddeSayisi += 1
      aramaYapilacakYerinBasi+=maddeninBaslayabilecegiYer+5
    maddeSayisi+=(geciciMaddeSayisi+ekMaddeSayisi+ekMaddeSayisi2+hataliMaddeSayisi)

    if df['madde_sayisi'][ilgiliKayit]==0:
      df.loc[ilgiliKayit,'madde_sayisi'] = maddeSayisi
    else:
      if maddeSayisi != df['madde_sayisi'][ilgiliKayit]:
        kategoridekiHataliBelgeSayisi[kategorilerimiz.index(df["kategori"][ilgiliKayit])] += 1
        print('Kayıt No:',ilgiliKayit,'Olması Gereken:',df['madde_sayisi'][ilgiliKayit],'Tahmin:',maddeSayisi,'Kategori:',df['kategori'][ilgiliKayit])
        if df['madde_sayisi'][ilgiliKayit]>maddeSayisi:
          maddeSayisiEksikSayilanBelgeSayisi += 1
        else:
          maddeSayisiFazlaSayilanBelgeSayisi += 1
        if abs(df['madde_sayisi'][ilgiliKayit]-maddeSayisi)==1:
          birFarkOlanBelgeSayisi += 1

for i in range(len(kategorilerimiz)):
  print(kategorilerimiz[i],':',kategoridekiHataliBelgeSayisi[i],'/',kategoridekiToplamBelgeSayisi[i],'=',round(kategoridekiHataliBelgeSayisi[i]/kategoridekiToplamBelgeSayisi[i],3))

print('Toplam Hata:',sum(kategoridekiHataliBelgeSayisi))

print('Madde sayısı eksik tahmin edilen belge sayısı:',maddeSayisiEksikSayilanBelgeSayisi)
print('Madde sayısı fazla tahmin edilen belge sayısı:',maddeSayisiFazlaSayilanBelgeSayisi)
print('Madde sayısı 1 madde fark ile yanlış tespit edilen belge sayısı:',birFarkOlanBelgeSayisi)

# **Çıkarım:**

- Toplam 221 hatanın 208 tanesinde madde sayısını eksik tahmin ediyoruz yani sayamadığımız maddeler var.
- 13 tane hatada da olmayan yani fazladan saydığımız madde var.
- Toplam 221 hatadan 90 tanesinde de 1 madde fark ile madde sayısını yanlış tespit ediyoruz.